In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Section 1: Web Scraper ###
Import the data from the webpage and save it as a new Beautiful Soup object _soup_

In [2]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(path)
assert(result.status_code == 200) # make sure the request came back positive from the server
soup = BeautifulSoup(result.content, 'html5lib')

In [3]:
table = soup.body.table
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighbourhood'])

rows = table.findAll('tr')
for i, row in enumerate(rows):
    values = [x.getText().rstrip('\n') for x in row.findAll('td')]
    if len(values) > 0 : df.loc[i] = values

Extract the DataFrame with postcodes from the resulting list _wiki_, do some formatting and remove '_Not assigned_' values from the Borough column

In [8]:
df = df[df['Borough'] != 'Borough']
df = df[df['Borough'] != 'Not assigned']
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.reset_index(drop=True, inplace=True)

codes = pd.DataFrame(df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)))
codes.reset_index(inplace=True) # Grouping the DataFrame made the columns Postcode and Borough the indices

### Section 2: Starting the Location Retrieval ###
I will use the provided .csv file

In [9]:
postal = pd.read_csv('https://cocl.us/Geospatial_data')
print(postal.shape)
postal.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
postal.columns = ['PostalCode', 'Latitude', 'Longitude']
locations = pd.merge(codes, postal, on='PostalCode', how='inner')
print(locations.shape)
locations.head(10)

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
